## <center>Spline-ok, B-splineok</center>



### 1. Különböző basis splineok ábrázolása

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, FloatSlider

In [2]:
def even_knots(n_internal, k0=0.0, kn=1.0):
    """
    Egyenletesen elosztott csomópontok [k0, kn] intervallumon.
    Visszaadja a knot-vektort (numpy array).
    """
    # n_internal belső csomópont + 2 szélső
    return np.linspace(k0, kn, n_internal + 2)


def bspline_basis(x, i, degree, knots):
    """
    Cox–de Boor rekurzióval kiszámolja az i-edik B-spline bázisfüggvény
    értékét a 'x' pontokon, adott fok (degree) és knot-vektor mellett.

    x      : numpy array
    i      : bázis indexe (lehet 0-tól len(knots)-2-degree-ig)
    degree : fok (0 = lépcső, 1 = lineáris, 2 = kvadratikus, 3 = kubikus, stb.)
    knots  : csomópontok (numpy array)
    """
    x = np.asarray(x)

    # 0. fok: indikátor az [k_i, k_{i+1}) intervallumra
    if degree == 0:
        left = knots[i]
        right = knots[i + 1]
        y = np.zeros_like(x, dtype=float)
        y[(x >= left) & (x < right)] = 1.0
        # utolsó intervallumnál vegyük bele a jobb szélt is
        if i + 1 == len(knots) - 1:
            y[x == right] = 1.0
        return y

    # magasabb fok: rekurzív definíció
    denom1 = knots[i + degree] - knots[i]
    denom2 = knots[i + degree + 1] - knots[i + 1]

    # w1, w2 súlyok (vigyázunk a 0 osztásokkal)
    if denom1 == 0:
        w1 = np.zeros_like(x, dtype=float)
    else:
        w1 = (x - knots[i]) / denom1

    if denom2 == 0:
        w2 = np.zeros_like(x, dtype=float)
    else:
        w2 = (knots[i + degree + 1] - x) / denom2

    y1 = bspline_basis(x, i,     degree - 1, knots)
    y2 = bspline_basis(x, i + 1, degree - 1, knots)

    return w1 * y1 + w2 * y2


In [3]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, Text

def plot_spline_with_custom_coeffs(degree=1, n_internal=4,
                                   coeffs_str="0.4, 0.4, 2.5, 3.1, 1.4, 1.4, 1, 1, 1, 1, 1"):
    """
    - degree: B-spline fok (0=lépcső, 1=lineáris, 2=kvadratikus, 3=kubikus)
    - n_internal: belső csomópontok száma
    - coeffs_str: vesszővel elválasztott súlyok, pl. "0.4, 0.4, 2.5"
    """

    # csomópontok
    knots = even_knots(n_internal=n_internal, k0=0.0, kn=1.0)

    # x-rács
    x = np.linspace(knots[0], knots[-1], 400)

    # bázisok száma
    n_basis = len(knots) - degree - 1
    if n_basis <= 0:
        print("Túl nagy fok ehhez a knots-számhoz, csökkentsd a degree-t vagy növeld n_internal-t!")
        return

    # szövegmezőből súlyok beolvasása
    try:
        coeffs_list = [float(s.strip()) for s in coeffs_str.split(",") if s.strip() != ""]
    except ValueError:
        print("Nem tudtam float-tá alakítani a súlyok egy részét. Írj olyasmit, mint: 0.4, 2.5, 1.0")
        return

    if len(coeffs_list) == 0:
        print("Nem adtál meg súlyokat.")
        return

    # ha kevesebb súly van, mint bázis, kipótoljuk nullákkal
    if len(coeffs_list) < n_basis:
        coeffs_list = coeffs_list + [0.0] * (n_basis - len(coeffs_list))
    # ha több, levágjuk
    coeffs = np.array(coeffs_list[:n_basis])

    # bázisfüggvények
    basis_vals = []
    for i in range(n_basis):
        y = bspline_basis(x, i, degree, knots)
        basis_vals.append(y)
    basis_vals = np.array(basis_vals)  # (n_basis, len(x))

    # spline(x) = Σ a_i B_i(x)
    spline_vals = (coeffs[:, None] * basis_vals).sum(axis=0)

    # -------------- ÁBRA ----------------
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4), sharex=True)

    # B-spline bázisok
    for i in range(n_basis):
        ax1.plot(x, basis_vals[i], label=f"B{i}(x)")
    ax1.set_title(f"{degree}. fokú B-spline bázisok\n(n_internal = {n_internal}, n_basis = {n_basis})")
    ax1.set_xlabel("x")
    ax1.set_ylabel("B_i(x)")
    ax1.grid(True)
    ax1.legend(fontsize=8)

    # spline
    ax2.plot(x, spline_vals, "r", label="spline(x) = Σ a_i B_i(x)")
    ax2.set_title("Súlyozott B-spline kombináció")
    ax2.set_xlabel("x")
    ax2.set_ylabel("spline(x)")
    ax2.grid(True)

    # csomópontok jelölése
    for k in knots:
        ax2.axvline(k, linestyle=":", linewidth=0.7)

    # ==== ÚJ RÉSZ: súlyfeliratok a bázisok csúcsán ====

    ymin, ymax = ax2.get_ylim()
    dy = 0.03 * (ymax - ymin)  # kis függőleges eltolás

    for i in range(n_basis):
        Bi = basis_vals[i]            # i-edik B-spline értékei
        j_max = np.argmax(Bi)         # ahol a legnagyobb
        x_pos = x[j_max]
        y_pos = spline_vals[j_max]

        ax2.text(
            x_pos,
            y_pos + dy,
            rf"$a_{i}={coeffs[i]:.2f}$",
            ha="center",
            va="bottom",
            color="teal",
            fontsize=9,
            bbox=dict(facecolor="white", alpha=0.7, edgecolor="none"),
        )

    ax2.legend()
    plt.tight_layout()
    plt.show()


interact(
    plot_spline_with_custom_coeffs,
    degree=IntSlider(value=1, min=0, max=3, step=1, description="fok"),
    n_internal=IntSlider(value=4, min=1, max=10, step=1, description="belső knots"),
    coeffs_str=Text(
        value="0.4, 0.4, 2.5, 3.1, 1.4, 1.4, 1, 1, 1, 1, 1",
        description="súlyok (a_i):",
        continuous_update=False
    ),
);


interactive(children=(IntSlider(value=1, description='fok', max=3), IntSlider(value=4, description='belső knot…

### 2. Sinx  közelítése B-splineokkal

In [4]:
import numpy as np
import matplotlib.pyplot as plt

def open_uniform_knots(degree, n_basis, a=0.0, b=1.0):
    """
    Open uniform B-spline knotvektor:
    - a bal szélen degree+1-szer ismételt 'a'
    - a jobb szélen degree+1-szer ismételt 'b'
    - közte egyenletes belső knotok (ha vannak)

    degree : fok (0,1,2,3,...)
    n_basis: B-spline bázisfüggvények száma
    [a,b]  : intervallum
    """
    # knotok száma: n_basis + degree + 1
    n_knots = n_basis + degree + 1
    knots = np.empty(n_knots)

    # bal szélső ismétlés
    knots[:degree+1] = a
    # jobb szélső ismétlés
    knots[-(degree+1):] = b

    # belső knotok (ha marad hely)
    if n_knots > 2*(degree+1):
        n_internal = n_knots - 2*(degree+1)
        # belső knotok egyenletesen [a,b]-n
        internal = np.linspace(a, b, n_internal+2)[1:-1]
        knots[degree+1:-degree-1] = internal

    return knots


In [5]:
# Céfüggvény – ezt nyugodtan átírhatod
def target(x):
    return np.sin(2 * np.pi * x) + 0.3 * x

# Tanítópontok
rng = np.random.default_rng(0)
x_train = np.linspace(0.0, 1.0, 200)
y_train = target(x_train)


In [6]:
def train_and_plot_by_basis(epochs=0, degree=3, n_basis_slider=4, lr=0.05):
    """
    epochs        : GD lépések száma
    degree        : B-spline fok
    n_basis_slider: B-spline bázisfüggvények száma
    lr            : learning rate
    """

    n_basis = n_basis_slider
    knots = open_uniform_knots(degree=degree, n_basis=n_basis, a=0.0, b=1.0)

    x = x_train
    y = y_train

    # Design-mátrix
    B = np.zeros((len(x), n_basis))
    for i in range(n_basis):
        B[:, i] = bspline_basis(x, i, degree, knots)

    # Súlyok (0-ról indul)
    a = np.zeros(n_basis)

    # Gradient descent
    for _ in range(epochs):
        y_pred = B @ a
        diff = y_pred - y
        grad = 2.0 / len(x) * (B.T @ diff)
        a -= lr * grad

    # végső loss
    y_pred = B @ a
    loss = np.mean((y_pred - y)**2)
    print(f"MSE loss = {loss:.6f} (epochs={epochs}, degree={degree}, n_basis={n_basis})")

    # kiértékelés tömör rácson
    x_plot = np.linspace(0.0, 1.0, 400)
    y_true_plot = target(x_plot)

    B_plot = np.zeros((len(x_plot), n_basis))
    for i in range(n_basis):
        B_plot[:, i] = bspline_basis(x_plot, i, degree, knots)
    y_spline_plot = B_plot @ a

    # ------- ÁBRA -------
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 4), sharex=True)

    # 1) B-spline bázisok
    colors = plt.cm.tab10(np.linspace(0, 1, n_basis))
    for i in range(n_basis):
        ax1.plot(x_plot, B_plot[:, i], color=colors[i], label=f"B{i}")
    ax1.set_title(f"{degree}. fokú open uniform B-spline bázisok\n(n_basis={n_basis})")
    ax1.set_xlabel("x")
    ax1.set_ylabel("B_i(x)")
    ax1.grid(True)

    # 2) Céfüggvény + spline + adatpontok
    ax2.scatter(x, y, s=10, alpha=0.4, label="tanítópontok")
    ax2.plot(x_plot, y_true_plot, "k--", label="céfüggvény")
    ax2.plot(x_plot, y_spline_plot, "r", label="spline közelítés")

    # függőleges vonalak a knotoknál
    for k in knots:
        ax2.axvline(k, linestyle=":", linewidth=0.7)

    ax2.set_xlabel("x")
    ax2.set_ylabel("y")
    ax2.set_title(f"Gradient descent spline tanulás\nMSE = {loss:.4f}")
    ax2.legend()
    ax2.grid(True)

    # -----> IDE ÍRJUK KI A SÚLYOKAT <-----
    # súlyok listázása a jobb felső sarokban
    weights_text = "\n".join([f"a[{i}] = {a[i]: .3f}" for i in range(n_basis)])
    ax2.text(
        1.02, 0.5,
        f"Súlyok:\n{weights_text}",
        transform=ax2.transAxes,
        fontsize=10,
        verticalalignment="center",
        bbox=dict(facecolor="white", alpha=0.7)
    )

    plt.tight_layout()
    plt.show()


In [ ]:
from ipywidgets import interact, IntSlider, FloatSlider

interact(
    train_and_plot_by_basis,
    epochs=IntSlider(value=0, min=0, max=200, step=10, description="epochs"),
    degree=IntSlider(value=3, min=0, max=3, step=1, description="fok"),
    n_basis_slider=IntSlider(value=4, min=2, max=10, step=1, description="B-spline-ok száma"),
    lr=FloatSlider(value=0.05, min=0.005, max=0.1, step=0.005, description="lr"),
);


interactive(children=(IntSlider(value=0, description='epochs', max=200, step=10), IntSlider(value=3, descripti…

### 3. Két függvény összegének közelítése B-splineokkal (részenként)

In [8]:
def open_uniform_knots(degree, n_basis, a=0.0, b=1.0):
    n_knots = n_basis + degree + 1
    knots = np.empty(n_knots)

    knots[:degree+1] = a
    knots[-(degree+1):] = b

    if n_knots > 2*(degree+1):
        n_internal = n_knots - 2*(degree+1)
        internal = np.linspace(a, b, n_internal+2)[1:-1]
        knots[degree+1:-degree-1] = internal

    return knots

def bspline_basis(x, i, degree, knots):
    x = np.asarray(x)

    if degree == 0:
        left, right = knots[i], knots[i+1]
        y = np.zeros_like(x, dtype=float)
        y[(x >= left) & (x < right)] = 1.0
        if i+1 == len(knots)-1:
            y[x == right] = 1.0
        return y

    denom1 = knots[i+degree]   - knots[i]
    denom2 = knots[i+degree+1] - knots[i+1]

    w1 = np.zeros_like(x, dtype=float) if denom1 == 0 else (x - knots[i]) / denom1
    w2 = np.zeros_like(x, dtype=float) if denom2 == 0 else (knots[i+degree+1] - x) / denom2

    y1 = bspline_basis(x, i,     degree-1, knots)
    y2 = bspline_basis(x, i + 1, degree-1, knots)

    return w1 * y1 + w2 * y2


In [9]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, FloatSlider

# fizikai tartomány: [0, 2π]
t_min, t_max = 0.0, 2*np.pi

# tanítópontok a fizikai tartományban
t_train = np.linspace(t_min, t_max, 200)

# spline-hez használt, [0,1]-re skálázott változó
x_train = (t_train - t_min) / (t_max - t_min)

# célfüggvény komponensek a fizikai tartományban
alpha = 0.5  # vagy 0.05, 0.01 – ízlés szerint
beta =  3

y_sin = np.sin(beta * t_train)
y_sq  = alpha * t_train**2
y_sum = y_sin + y_sq

In [10]:
def train_and_plot_two_splines(epochs=0, degree=3, n_basis_slider=7, lr=0.01):
    """
    Két spline ugyanazon B-spline bázison:
      s1(x) ~ sin(t)
      s2(x) ~ t^2
      s1+s2 ~ sin(t)+t^2

    Itt x a [0,1]-re skálázott t.
    """
    n_basis = n_basis_slider
    knots = open_uniform_knots(degree=degree, n_basis=n_basis, a=0.0, b=1.0)

    # Design-mátrix [0,1]-re skálázott x_train-en
    N = len(x_train)
    B = np.zeros((N, n_basis))
    for i in range(n_basis):
        B[:, i] = bspline_basis(x_train, i, degree, knots)

    # két súlyvektor
    a_sin = np.zeros(n_basis)
    a_sq  = np.zeros(n_basis)

    # gradient descent mindkét komponensre
    for _ in range(epochs):
        # sin komponens
        pred_sin = B @ a_sin
        diff_sin = pred_sin - y_sin
        grad_sin = 2.0 / N * (B.T @ diff_sin)
        a_sin -= lr * grad_sin

        # t^2 komponens
        pred_sq = B @ a_sq
        diff_sq = pred_sq - y_sq
        grad_sq = 2.0 / N * (B.T @ diff_sq)
        a_sq -= lr * grad_sq

    # végső predikciók tanítópontokon
    pred_sin = B @ a_sin
    pred_sq  = B @ a_sq
    pred_sum = pred_sin + pred_sq

    mse_sin = np.mean((pred_sin - y_sin)**2)
    mse_sq  = np.mean((pred_sq  - y_sq)**2)
    mse_sum = np.mean((pred_sum - y_sum)**2)
    print(f"MSE_sin = {mse_sin:.5f}, MSE_t2 = {mse_sq:.5f}, MSE_total = {mse_sum:.5f}")

    # sűrű rács a fizikai tartományon
    t_plot = np.linspace(t_min, t_max, 400)
    x_plot = (t_plot - t_min) / (t_max - t_min)

    B_plot = np.zeros((len(x_plot), n_basis))
    for i in range(n_basis):
        B_plot[:, i] = bspline_basis(x_plot, i, degree, knots)

    s1_plot   = B_plot @ a_sin
    s2_plot   = B_plot @ a_sq
    s_sum_plot = s1_plot + s2_plot

    sin_plot = np.sin(beta * t_plot)
    t2_plot  = alpha * t_plot**2
    sum_plot = sin_plot + t2_plot

    # ---- ÁBRÁK ----
    fig, axes = plt.subplots(2, 2, figsize=(14, 8), sharex=False)
    ax_basis, ax_sin, ax_sq, ax_sum = axes.flatten()

    # 1) B-spline bázisok [0,1]-en (x tengely!)
    colors = plt.cm.tab10(np.linspace(0, 1, n_basis))
    for i in range(n_basis):
        ax_basis.plot(x_plot, B_plot[:, i], color=colors[i])
    ax_basis.set_title(f"{degree}. fokú B-spline bázisok (n_basis={n_basis})\n(x skálázott ∈ [0,1])")
    ax_basis.set_xlabel("x (skálázott)")
    ax_basis.set_ylabel("B_i(x)")
    ax_basis.grid(True)

    # 2) sin(t) komponens – t tengelyen
    ax_sin.plot(t_plot, sin_plot, "k--", label="sin(t)")
    ax_sin.plot(t_plot, s1_plot, "r", label="spline₁(t) ≈ sin(t)")
    ax_sin.set_title(f"sin(t) komponens (MSE={mse_sin:.4f})")
    ax_sin.legend()
    ax_sin.grid(True)

    # 3) t² komponens
    ax_sq.plot(t_plot, t2_plot, "k--", label="t²")
    ax_sq.plot(t_plot, s2_plot, "g", label="spline₂(t) ≈ t²")
    ax_sq.set_title(f"t² komponens (MSE={mse_sq:.4f})")
    ax_sq.legend()
    ax_sq.grid(True)

    # 4) összeg
    ax_sum.plot(t_plot, sum_plot, "k--", label="sin(t)+t²")
    ax_sum.plot(t_plot, s_sum_plot, "b", label="spline₁+spline₂")
    ax_sum.set_title(f"Összeg (MSE={mse_sum:.4f})")
    ax_sum.legend()
    ax_sum.grid(True)

    for ax in (ax_sin, ax_sq, ax_sum):
        ax.set_xlabel("t")

    plt.tight_layout()
    plt.show()


In [ ]:
interact(
    train_and_plot_two_splines,
    epochs=IntSlider(value=0, min=0, max=400, step=20, description="epochs"),
    degree=IntSlider(value=3, min=0, max=3, step=1, description="fok"),
    n_basis_slider=IntSlider(value=7, min=3, max=20, step=1, description="B-spline-ok"),
    lr=FloatSlider(value=0.01, min=0.001, max=0.1, step=0.001, description="lr"),
);

interactive(children=(IntSlider(value=0, description='epochs', max=400, step=20), IntSlider(value=3, descripti…

### 4. Két függvény összegének közelítése B-splineokkal (egyszerre)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, FloatSlider

# ===== 1. B-spline alapok =====

def open_uniform_knots(degree, n_basis, a=0.0, b=1.0):
    """
    Open uniform B-spline knotvektor:
    - bal szélen degree+1-szer 'a'
    - jobb szélen degree+1-szer 'b'
    - közte egyenletes belső knotok (ha vannak)
    """
    n_knots = n_basis + degree + 1
    knots = np.empty(n_knots)

    knots[:degree+1] = a
    knots[-(degree+1):] = b

    if n_knots > 2*(degree+1):
        n_internal = n_knots - 2*(degree+1)
        internal = np.linspace(a, b, n_internal+2)[1:-1]
        knots[degree+1:-degree-1] = internal

    return knots


def bspline_basis(x, i, degree, knots):
    """
    Cox–de Boor B-spline bázisfüggvény.
    """
    x = np.asarray(x)

    if degree == 0:
        left, right = knots[i], knots[i+1]
        y = np.zeros_like(x, dtype=float)
        y[(x >= left) & (x < right)] = 1.0
        if i+1 == len(knots)-1:
            y[x == right] = 1.0
        return y

    denom1 = knots[i+degree]   - knots[i]
    denom2 = knots[i+degree+1] - knots[i+1]

    w1 = np.zeros_like(x, dtype=float) if denom1 == 0 else (x - knots[i]) / denom1
    w2 = np.zeros_like(x, dtype=float) if denom2 == 0 else (knots[i+degree+1] - x) / denom2

    y1 = bspline_basis(x, i,     degree-1, knots)
    y2 = bspline_basis(x, i + 1, degree-1, knots)

    return w1 * y1 + w2 * y2


# ===== 2. Tanító adatok a "fizikai" tartományban =====

# fizikai tartomány: t ∈ [0, 2π]
t_min, t_max = 0.0, 2*np.pi
t_train = np.linspace(t_min, t_max, 300)

# [0,1]-re skálázott változó a spline-hoz
x_train = (t_train - t_min) / (t_max - t_min)

# függvény: f(t) = sin(t) + alpha * t^2
alpha = 0.1  # hogy a sin komponens is látszódjon
y_true = np.sin(t_train) + alpha * t_train**2


# ===== 3. Közös lossból tanuló két spline =====

def train_and_plot_common_loss(epochs=0, degree=3, n_basis_slider=7, lr=0.01):
    """
    Két spline aktiváció (s1, s2) ugyanazon B-spline bázison.
    Loss csak az összegre:
        L = mean( (s1(t) + s2(t) - f(t))^2 )

    epochs        : GD lépések száma
    degree        : B-spline fok
    n_basis_slider: B-spline bázisok száma
    lr            : tanulási ráta
    """
    n_basis = n_basis_slider
    knots = open_uniform_knots(degree=degree, n_basis=n_basis, a=0.0, b=1.0)

    # design-mátrix a skálázott x_train-en
    N = len(x_train)
    B = np.zeros((N, n_basis))
    for i in range(n_basis):
        B[:, i] = bspline_basis(x_train, i, degree, knots)

    # két súlyvektor - KIS véletlen zajjal, hogy ne legyen teljesen szimmetrikus
    rng = np.random.default_rng(0)
    a1 = 0.1 * rng.normal(size=n_basis)
    a2 = 0.1 * rng.normal(size=n_basis)

    # gradient descent - KÖZÖS loss
    for _ in range(epochs):
        s1 = B @ a1
        s2 = B @ a2
        y_pred = s1 + s2
        diff = y_pred - y_true   # közös hiba

        # ∂L/∂a1 = B^T diff * (2/N), ugyanaz a2-re is, mert y = B a1 + B a2
        grad = 2.0 / N * (B.T @ diff)
        a1 -= lr * grad
        a2 -= lr * grad

    # végső predikciók tanítópontokon
    s1 = B @ a1
    s2 = B @ a2
    y_pred = s1 + s2

    mse_total = np.mean((y_pred - y_true)**2)

    # Csak vizualizáció kedvéért külön MSE-k a "valódi" komponensekre:
    y_sin = np.sin(t_train)
    y_t2  = alpha * t_train**2
    mse_sin_comp = np.mean((s1 - y_sin)**2)
    mse_t2_comp  = np.mean((s2 - y_t2)**2)

    print(f"MSE_total = {mse_total:.5f},   MSE_spline1~sin = {mse_sin_comp:.5f},   MSE_spline2~alpha*t² = {mse_t2_comp:.5f}")

    # sűrű rács a fizikai tartományon
    t_plot = np.linspace(t_min, t_max, 400)
    x_plot = (t_plot - t_min) / (t_max - t_min)

    B_plot = np.zeros((len(x_plot), n_basis))
    for i in range(n_basis):
        B_plot[:, i] = bspline_basis(x_plot, i, degree, knots)

    s1_plot = B_plot @ a1
    s2_plot = B_plot @ a2
    y_pred_plot = s1_plot + s2_plot

    sin_plot = np.sin(t_plot)
    t2_plot  = alpha * t_plot**2
    sum_plot = sin_plot + t2_plot

    # ===== ÁBRÁK =====
    fig, axes = plt.subplots(2, 2, figsize=(14, 8))
    ax_basis, ax_s1, ax_s2, ax_sum = axes.flatten()

    # 1) B-spline bázisok (x ∈ [0,1])
    colors = plt.cm.tab10(np.linspace(0, 1, n_basis))
    for i in range(n_basis):
        ax_basis.plot(x_plot, B_plot[:, i], color=colors[i])
    ax_basis.set_title(f"{degree}. fokú B-spline bázisok (n_basis={n_basis})\n x skálázott ∈ [0,1]")
    ax_basis.set_xlabel("x")
    ax_basis.set_ylabel("B_i(x)")
    ax_basis.grid(True)

    # 2) s1 komponens vs sin(t)
    ax_s1.plot(t_plot, sin_plot, "k--", label="sin(t)")
    ax_s1.plot(t_plot, s1_plot, "r", label="spline₁(t)")
    ax_s1.set_title(f"spline₁ vs sin(t)  (CSAK VIZUALIZÁCIÓ, loss nem ezt használja)\nMSE≈{mse_sin_comp:.4f}")
    ax_s1.legend()
    ax_s1.grid(True)

    # 3) s2 komponens vs alpha * t²
    ax_s2.plot(t_plot, t2_plot, "k--", label=f"{alpha}·t²")
    ax_s2.plot(t_plot, s2_plot, "g", label="spline₂(t)")
    ax_s2.set_title(f"spline₂ vs {alpha}·t²  (CSAK VIZUALIZÁCIÓ)\nMSE≈{mse_t2_comp:.4f}")
    ax_s2.legend()
    ax_s2.grid(True)

    # 4) összeg – EZRE van a loss
    ax_sum.plot(t_plot, sum_plot, "k--", label="sin(t)+αt²")
    ax_sum.plot(t_plot, y_pred_plot, "b", label="spline₁+spline₂")
    ax_sum.set_title(f"Összeg (valódi loss)  MSE_total≈{mse_total:.4f}")
    ax_sum.legend()
    ax_sum.grid(True)

    for ax in (ax_s1, ax_s2, ax_sum):
        ax.set_xlabel("t")

    plt.tight_layout()
    plt.show()


# ===== 4. Interaktív csúszkák =====

interact(
    train_and_plot_common_loss,
    epochs=IntSlider(value=0, min=0, max=400, step=20, description="epochs"),
    degree=IntSlider(value=3, min=0, max=3, step=1, description="fok"),
    n_basis_slider=IntSlider(value=7, min=3, max=20, step=1, description="B-spline-ok"),
    lr=FloatSlider(value=0.01, min=0.001, max=0.1, step=0.001, description="lr"),
);


interactive(children=(IntSlider(value=0, description='epochs', max=400, step=20), IntSlider(value=3, descripti…